# Auto ML Regressor Predictions

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from autosklearn.regression import AutoSklearnRegressor
from scipy import stats
import utils as u
import copy

import matplotlib.pylab as plt
plt.style.use('ggplot')

def root_mean_squared_log_error(y_true, y_pred):
    # Alternatively: sklearn.metrics.mean_squared_log_error(y_true, y_pred) ** 0.5
    assert (y_true >= 0).all() 
    assert (y_pred >= 0).all()
    log_error = np.log1p(y_pred) - np.log1p(y_true)  # Note: log1p(x) = log(1 + x)
    return np.mean(log_error ** 2) ** 0.5

%run utils.py
%run ../common_utils.py

ModuleNotFoundError: No module named 'autosklearn'

In [ ]:
train, test, metaData = load_all_data()
train, test = polar_coordinates(train, test)

In [ ]:
###### MISSING VALUES
# area_living
train = u.fillnaReg(train, ['area_total'], 'area_living')

# area_kitchen
train = u.fillnaReg(train, ['area_total', 'area_living'], 'area_kitchen')

# ceiling
train = u.fillnaReg(train, ['stories'], 'ceiling')

In [ ]:
############## SETTINGS
test_size = 0.2
random_state_split = 42
features = ['area_total','floor','bathrooms_shared','district','constructed','material','stories', 'r', 'theta']

In [ ]:
train.fillna(train.mean(), inplace = True)
test.fillna(test.mean(), inplace = True)
train_data, test_data = train_test_split(train, test_size=test_size, random_state=random_state_split, stratify=np.log(train.price).round())

In [ ]:
X_train, y_train = train_data[features], train_data['price']
X_test, y_test = test_data[features], test_data['price']

In [ ]:
model = AutoSklearnRegressor()

In [ ]:
model.fit(X_train, y_train)
rf_prediction = model.predict(X_test)
rf_mse = root_mean_squared_log_error(rf_prediction, y_test)
print('Test RMSLE:', rf_mse)
plt.scatter(rf_prediction, y_test, alpha=0.2)
plt.xlabel('Random Forest prediction')
plt.ylabel('Ground Truth')